In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json

--2021-10-28 22:43:12--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.192.128, 209.85.145.128, 172.217.219.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.192.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.05s   

2021-10-28 22:43:12 (107 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



# Loading the dataset

In [2]:
!pip install simplejson

     |████████████████████████████████| 129 kB 5.5 MB/s 


In [3]:
import simplejson as json

In [4]:
training_size = 20000
with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)
sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
training_sentences = sentences[0:training_size]
validation_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
validation_labels = labels[training_size:]

In [5]:
training_sentences[0:5]

["former versace store clerk sues over secret 'black code' for minority shoppers",
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
 "mom starting to fear son's web series closest thing she will have to grandchild",
 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
 'j.k. rowling wishes snape happy birthday in the most magical way']

In [6]:
training_labels[0:5]

[0, 0, 1, 1, 0]

In [7]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 48.0 MB/s 
     |████████████████████████████████| 895 kB 49.9 MB/s 
     |████████████████████████████████| 3.3 MB 46.1 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf

# Tokenizing the dataset using distilbert-base-uncased

In [9]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [10]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

# Definind the model for training

In [12]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

#Training using the model 'distilbert-base-uncased'

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=3,
          batch_size=16,
          callbacks=[tensorboard_callback],verbose=1,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/3
1250/1250 [==============================] - 348s 269ms/step - loss: 0.2641 - accuracy: 0.8849 - val_loss: 0.2049 - val_accuracy: 0.9183
Epoch 2/3
1250/1250 [==============================] - 334s 267ms/step - loss: 0.0898 - accuracy: 0.9678 - val_loss: 0.2444 - val_accuracy: 0.9220
Epoch 3/3
1250/1250 [==============================] - 336s 268ms/step - loss: 0.0407 - accuracy: 0.9860 - val_loss: 0.3487 - val_accuracy: 0.9058


In [15]:
from tensorboard import notebook
notebook.display(port=6006, height=1000)

<IPython.core.display.Javascript object>

In [16]:
! pip install gradio

     |████████████████████████████████| 2.0 MB 5.2 MB/s 
     |████████████████████████████████| 206 kB 44.7 MB/s 
     |████████████████████████████████| 1.9 MB 33.9 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 3.5 MB 52.0 MB/s 
     |████████████████████████████████| 961 kB 45.6 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4710 sha256=a249bd805b2747165abd47415e4aff7e7633b24333440f845ac236ea19435354
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3371 sha256=a031d3ae22e6a280d8e0c01bf37c97bb044f485825c6ed420240d859ddd6e3af
  Stored in directory: /root/.cache/pip/wheels/28/c0/c4/44687421dab41455be93112bd1b0dee1f3c5a9aa27bee63708
Successfully built ffmpy flask-cachebuster


#Simple component built with gradio

In [17]:
import gradio
def get_prediction(text):
  def get_text(prediction):
    if prediction_probs[0][1] > prediction_probs[0][0]:
      return "Sarcastic"
    else:
      return "Not Sarcastic"
  tokenized_text=tokenizer.encode(text,truncation=True,padding=True,return_tensors="tf")
  prediction=model(tokenized_text)
  prediction_logits = prediction[0]
  prediction_probs = tf.nn.softmax(prediction_logits,axis=1).numpy()
  return get_text(prediction_probs)

iface = gradio.Interface(fn=get_prediction, inputs="text", outputs=["text"])
iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://55085.gradio.app


KeyboardInterrupt: ignored